## import necessary packages

In [1]:
from pymongo import MongoClient
import pandas as pd

## Import weather data and convert to DataFrame

### access mongo db

In [2]:
client = MongoClient()

In [3]:
ferry_db = client['ferry']

In [4]:
weather = ferry_db['weather']

### examine record structure for weather

In [5]:
weather.find_one()

{'_id': ObjectId('5e86586d17b20f91ad7ec965'),
 'latitude': 47.811784,
 'longitude': -122.38325,
 'timezone': 'America/Los_Angeles',
 'daily': {'data': [{'time': 1481961600,
    'summary': 'Overcast throughout the day.',
    'icon': 'partly-cloudy-day',
    'sunriseTime': 1481990100,
    'sunsetTime': 1482020400,
    'moonPhase': 0.66,
    'precipIntensity': 0,
    'precipIntensityMax': 0,
    'precipIntensityMaxTime': 1482012060,
    'precipProbability': 0,
    'temperatureHigh': 33.3,
    'temperatureHighTime': 1482017880,
    'temperatureLow': 27.46,
    'temperatureLowTime': 1482037440,
    'apparentTemperatureHigh': 32.8,
    'apparentTemperatureHighTime': 1482017880,
    'apparentTemperatureLow': 27.95,
    'apparentTemperatureLowTime': 1482037440,
    'dewPoint': 21.51,
    'humidity': 0.76,
    'pressure': 1029.9,
    'windSpeed': 1.15,
    'windGust': 3.24,
    'windGustTime': 1482005460,
    'windBearing': 95,
    'cloudCover': 0.76,
    'uvIndex': 1,
    'uvIndexTime': 148200

### Get all records, convert to a list

In [6]:
weather_data = list(weather.find())

### Flatten information and add to weather_df

In [7]:
weather_df = pd.DataFrame()

In [8]:
for day in weather_data:
    day_df = pd.json_normalize(day['daily']['data'][0])
    day_df['date'] = day['date']
    weather_df = weather_df.append(day_df)

In [9]:
weather_df = weather_df.drop(columns=['time'])
weather_df.head()

,summary,icon,sunriseTime,sunsetTime,moonPhase,precipIntensity,precipIntensityMax,precipIntensityMaxTime,precipProbability,temperatureHigh,...,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime,date,precipType,precipAccumulation,ozone
0,Overcast throughout the day.,partly-cloudy-day,1481990100,1482020400,0.66,0.0000,0.0000,1.482012e+09,0.00,33.30,...,33.30,1482017880,23.35,1481972100,32.80,1482017880,2016-12-17,NaN,NaN,NaN
0,Drizzle overnight.,partly-cloudy-day,1482076560,1482106800,0.69,0.0000,0.0004,1.482131e+09,0.03,39.69,...,39.99,1482130800,29.76,1482051540,39.19,1482102060,2016-12-18,NaN,NaN,NaN
0,Light rain throughout the day.,rain,1482162960,1482193200,0.72,0.0124,0.0826,1.482214e+09,0.91,43.78,...,43.82,1482205260,32.26,1482155460,43.32,1482205260,2016-12-19,rain,NaN,NaN
0,Partly cloudy throughout the day.,rain,1482249420,1482279660,0.75,0.0037,0.0634,1.482221e+09,0.91,49.78,...,49.78,1482273900,39.37,1482303600,47.56,1482276840,2016-12-20,rain,NaN,NaN
0,Partly cloudy throughout the day.,partly-cloudy-day,1482335880,1482366060,0.78,0.0002,0.0007,1.482353e+09,0.10,45.85,...,45.85,1482357600,36.86,1482324360,45.35,1482357600,2016-12-21,rain,NaN,NaN


## Combine weather data with wait data

### Import wait data

In [10]:
ed_wait_df = pd.read_csv('../data/expanded/edmonds.csv')

In [11]:
ed_wait_df.head()

,time,tweet_text,wait_time,year,month,day,hour,dayofyear,week,weekday
0,2016-12-24 05:00:00-08:00,NaN,0.0,2016,12,24,5,359,51,5
1,2016-12-24 06:00:00-08:00,NaN,0.0,2016,12,24,6,359,51,5
2,2016-12-24 07:00:00-08:00,NaN,0.0,2016,12,24,7,359,51,5
3,2016-12-24 08:00:00-08:00,NaN,0.0,2016,12,24,8,359,51,5
4,2016-12-24 09:00:00-08:00,NaN,0.0,2016,12,24,9,359,51,5


### Prep data frames for merging w/ datetime adjustments

In [13]:
ed_wait_df['time'] = pd.to_datetime(ed_wait_df['time'], utc=True)
ed_wait_df['time'] = ed_wait_df['time'].dt.tz_convert('US/Pacific')

In [19]:
weather_df['date'] = weather_df['date'].dt.tz_localize('US/Pacific')

,summary,icon,sunriseTime,sunsetTime,moonPhase,precipIntensity,precipIntensityMax,precipIntensityMaxTime,precipProbability,temperatureHigh,...,temperatureMax,temperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,apparentTemperatureMax,apparentTemperatureMaxTime,date,precipType,precipAccumulation,ozone
0,Overcast throughout the day.,partly-cloudy-day,1481990100,1482020400,0.66,0.0000,0.0000,1.482012e+09,0.00,33.30,...,33.30,1482017880,23.35,1481972100,32.80,1482017880,2016-12-17 00:00:00-08:00,NaN,NaN,NaN
0,Drizzle overnight.,partly-cloudy-day,1482076560,1482106800,0.69,0.0000,0.0004,1.482131e+09,0.03,39.69,...,39.99,1482130800,29.76,1482051540,39.19,1482102060,2016-12-18 00:00:00-08:00,NaN,NaN,NaN
0,Light rain throughout the day.,rain,1482162960,1482193200,0.72,0.0124,0.0826,1.482214e+09,0.91,43.78,...,43.82,1482205260,32.26,1482155460,43.32,1482205260,2016-12-19 00:00:00-08:00,rain,NaN,NaN
0,Partly cloudy throughout the day.,rain,1482249420,1482279660,0.75,0.0037,0.0634,1.482221e+09,0.91,49.78,...,49.78,1482273900,39.37,1482303600,47.56,1482276840,2016-12-20 00:00:00-08:00,rain,NaN,NaN
0,Partly cloudy throughout the day.,partly-cloudy-day,1482335880,1482366060,0.78,0.0002,0.0007,1.482353e+09,0.10,45.85,...,45.85,1482357600,36.86,1482324360,45.35,1482357600,2016-12-21 00:00:00-08:00,rain,NaN,NaN


### Merge weather with wait time dataframe

In [20]:
ed_wait_df = pd.merge_asof(ed_wait_df, weather_df, left_on='time', right_on='date')

In [21]:
ed_wait_df.head().T

,0,1,2,3,4
time,2016-12-24 05:00:00-08:00,2016-12-24 06:00:00-08:00,2016-12-24 07:00:00-08:00,2016-12-24 08:00:00-08:00,2016-12-24 09:00:00-08:00
tweet_text,NaN,NaN,NaN,NaN,NaN
wait_time,0,0,0,0,0
year,2016,2016,2016,2016,2016
month,12,12,12,12,12
day,24,24,24,24,24
hour,5,6,7,8,9
dayofyear,359,359,359,359,359
week,51,51,51,51,51
weekday,5,5,5,5,5


## Add holiday data

In [56]:
holidays_df = pd.read_csv('../data/holidays.csv', header=None, names=['holiday'])

In [59]:
holidays_df['holiday'] = pd.to_datetime(holidays_df['holiday'])
holidays_df['holiday'] = holidays_df['holiday'].dt.tz_localize('US/Pacific')

In [60]:
holidays_df

,holiday
0,2016-01-01 00:00:00-08:00
1,2016-01-18 00:00:00-08:00
2,2016-02-15 00:00:00-08:00
3,2016-05-30 00:00:00-07:00
4,2016-07-04 00:00:00-07:00
5,2016-09-05 00:00:00-07:00
6,2016-11-11 00:00:00-08:00
7,2016-11-24 00:00:00-08:00
8,2016-12-26 00:00:00-08:00
9,2017-01-02 00:00:00-08:00


In [47]:
ed_wait_df = ed_wait_df.set_index(['time'])

In [ ]:
from datetime import timedelta

In [79]:
ed_wait_df['holiday'] = False
for holiday in holidays_df['holiday']:
    print(holiday)
    start = holiday - timedelta(days=0)
    end = holiday + timedelta(days=1)
    print(f"{start}  {end}")
    ed_wait_df.loc[start:end, 'holiday'] = True

2016-01-01 00:00:00-08:00
2016-01-01 00:00:00-08:00  2016-01-02 00:00:00-08:00
2016-01-18 00:00:00-08:00
2016-01-18 00:00:00-08:00  2016-01-19 00:00:00-08:00
2016-02-15 00:00:00-08:00
2016-02-15 00:00:00-08:00  2016-02-16 00:00:00-08:00
2016-05-30 00:00:00-07:00
2016-05-30 00:00:00-07:00  2016-05-31 00:00:00-07:00
2016-07-04 00:00:00-07:00
2016-07-04 00:00:00-07:00  2016-07-05 00:00:00-07:00
2016-09-05 00:00:00-07:00
2016-09-05 00:00:00-07:00  2016-09-06 00:00:00-07:00
2016-11-11 00:00:00-08:00
2016-11-11 00:00:00-08:00  2016-11-12 00:00:00-08:00
2016-11-24 00:00:00-08:00
2016-11-24 00:00:00-08:00  2016-11-25 00:00:00-08:00
2016-12-26 00:00:00-08:00
2016-12-26 00:00:00-08:00  2016-12-27 00:00:00-08:00
2017-01-02 00:00:00-08:00
2017-01-02 00:00:00-08:00  2017-01-03 00:00:00-08:00
2017-01-16 00:00:00-08:00
2017-01-16 00:00:00-08:00  2017-01-17 00:00:00-08:00
2017-02-20 00:00:00-08:00
2017-02-20 00:00:00-08:00  2017-02-21 00:00:00-08:00
2017-05-29 00:00:00-07:00
2017-05-29 00:00:00-07:00 

In [80]:
ed_wait_df['holiday'].head(480)

time
2016-12-24 05:00:00-08:00    False
2016-12-24 06:00:00-08:00    False
2016-12-24 07:00:00-08:00    False
2016-12-24 08:00:00-08:00    False
2016-12-24 09:00:00-08:00    False
                             ...  
2017-01-15 18:00:00-08:00    False
2017-01-15 19:00:00-08:00    False
2017-01-15 20:00:00-08:00    False
2017-01-15 21:00:00-08:00    False
2017-01-15 22:00:00-08:00    False
Name: holiday, Length: 480, dtype: bool

In [82]:
ed_wait_df.loc[start:end, 'holiday']

Series([], Name: holiday, dtype: bool)